# Collecting Tweets using Tweepy

In [1]:
# Load packages
import tweepy as tw #make sure to install v3.10
import pandas as pd
import numpy as np
# install preprocessor using pip install via Command Prompt/Anaconda prompt: pip install tweet-preprocessor
from preprocessor import api as p
#from preprocessor.api import clean, tokenize, parse #to clean tweets

In [2]:
# Credentials
access_token = "1311658729201377281-alEuRquK89jGbga7ckTsFhIlkRVPOY"
access_token_secret = "rpQowQaio69m3XMByxXEzUdo73WPnpaPuYTQpAFH9cjoa"
api_key = "E0RK2oHynrMIJ9YVrvuhIIjc6"
api_key_secret = "kHnfmtYn7J66MkR7bUE3BTnWsW64ZUDUIxtw9bThP5kmDjq0Vl"

In [3]:
# Authentication app
auth = tw.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

In [53]:
# Find by phrase or keywords within a specified time
search_words = '"jordan lake" has:links'
#search_words = '"jordan lake" -has:links' 
# we can not use -filter:links etc for premium API search
# use this link instead https://developer.twitter.com/en/docs/twitter-api/premium/rules-and-filtering/using-premium-operators
# it is equivalent to utah lake until:2020-12-20 since:2020-11-28 -filter:links
# we can't filter out retweets in the premium search

# advanced search in Twitter search box 
# utah lake (#utahlake) until:2020-12-03 since:2020-11-28 -filter:links -filter:retweets
# If I include hashtag in search, it wouldn't show the ones without hashtag
# May be we can have a separate search by #utahlake and then combine both the dataframes

# From date, to date, for premium API search the date should be in format yyyyMMddHHmm
year_from = int(input("Year from: "))
year_to = int(input("Year to: "))
date_since = f'{year_from}01010000' #from January 1st, 12:00AM UTC
date_until = f'{year_to}12312359' #to December 31st, 11:59PM UTC
#date_until = f'{year_to}12310300' #for 2020 3AM UTC

#date_since = f'{year}12250000' #test
#date_until = f'{year}12280000' #test

# Create a cursor object that iterates over the requested information
# had to create a dev environment named "SentimentAnalysis" in Twitter's Developer Portal
tweets = tw.Cursor(api.search_full_archive, environment_name = "SentimentAnalysis",
                   query=search_words,
                   fromDate=date_since, toDate=date_until).items()

# Parse the cursor object into rows of lists using list comprehension
#rows = [[tweet.user.location, tweet.text, tweet.created_at, tweet.coordinates] for tweet in tweets]

# Make a list to store each item return in a payload
rows = []

# Iterate through each object and get location, tweet, date, coordinates and place for each record
for tweet in tweets:
    
    # check if there is an 'extended_tweet' property
    if ('extended_tweet' in dir(tweet)) == True: 
        # if so, instead of 'text', get 'full_text'
        row = [tweet.user.location, tweet.extended_tweet['full_text'], tweet.created_at, tweet.coordinates, tweet.retweeted, tweet.place]
    else:
        row = [tweet.user.location, tweet.text, tweet.created_at, tweet.coordinates, tweet.retweeted, tweet.place]

    # Append each item to the list
    rows.append(row)

# Tweets starting with an RT mean that they are retweets 

# more on geo-referenced tweets: 
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/geo
# if required, look into user.derived.locations.geo
# it assigns lat/long based on user's profile and the geographic precision they set
# for example, if a user set their precision at the country-level, it will show center of US at lat/long
# so it is not too accurate and might be irrelevant to us
# https://developer.twitter.com/en/docs/twitter-api/enterprise/enrichments/overview/profile-geo

Year from: 2013
Year to: 2016


In [54]:
# Create dataframe from the rows of lists
df_tweet = pd.DataFrame(data=rows, columns=['location', 'tweet', 'time', 'coordinates', 'retweeted', 'place'])

In [55]:
# Add ID column (because we removed user information)
unique_id = np.arange(len(df_tweet))
df_tweet['id'] = unique_id + 1
df_tweet = df_tweet[['id', 'location', 'tweet', 'time', 'coordinates', 'retweeted', 'place']]

In [56]:
df_tweet.head(50)

,id,location,tweet,time,coordinates,retweeted,place
0,1,"North Carolina, USA","New Year's Day hike, bald eagle events kick of...",2016-12-31 15:38:03,None,False,None
1,2,"Tennessee, USA","Happy birthday to our lead player, Jordan Lake...",2016-12-31 14:14:03,None,False,None
2,3,"Raleigh, North Carolina",Jordan Lake getting in on the #SuperSunset act...,2016-12-31 03:08:34,None,False,None
3,4,North Carolina,RT @PittsboroCVB: #NewYearsDay hike at Jordan ...,2016-12-30 17:37:59,None,False,None
4,5,"North Carolina, USA",#NewYearsDay hike at Jordan Lake State Recreat...,2016-12-30 16:17:01,None,False,None
5,6,"Raleigh, Wake Forest, Clayton","If your New Year plans include a new home, the...",2016-12-30 13:45:21,None,False,None
6,7,None,"Jordan Lake, another great location for local ...",2016-12-30 12:09:41,None,False,None
7,8,"Los Angeles, CA",My beautiful #NorthCarolina @ Jordan Lake Stat...,2016-12-29 22:06:14,"{'type': 'Point', 'coordinates': [-79.0169, 35...",False,Place(_api=<tweepy.api.API object at 0x000002E...
8,9,Carrboro,#bestof2016 #eveningcommute \nsunsets over J...,2016-12-29 20:04:56,None,False,None
9,10,"California, USA",My beginning Gallery Wall. @ Legacy At Jordan ...,2016-12-28 23:33:19,"{'type': 'Point', 'coordinates': [-79.05953229...",False,Place(_api=<tweepy.api.API object at 0x000002E...


In [57]:
df_tweet.tail()

,id,location,tweet,time,coordinates,retweeted,place
10674,10675,"Grand Rapids, Mich.",A water search is underway in Jordan Lake at t...,2013-01-03 19:02:14,None,False,None
10675,10676,None,#RDU #Rent OWN a HOUSE FOR THE PRICE OF A USED...,2013-01-03 11:54:36,None,False,None
10676,10677,USA,Jordan Lake State Park - Plan An Ideal Weekend...,2013-01-02 01:31:15,None,False,None
10677,10678,Raleigh,Drinking a Double Naked Fish by @DuClawBrewing...,2013-01-01 21:47:26,"{'type': 'Point', 'coordinates': [-79.0081, 35...",False,Place(_api=<tweepy.api.API object at 0x000002E...
10678,10679,Raleigh,Drinking a Petrus Winter #9 by @BavikBeers @ S...,2013-01-01 06:55:00,"{'type': 'Point', 'coordinates': [-79.0081, 35...",False,Place(_api=<tweepy.api.API object at 0x000002E...


In [58]:
df_tweet.tweet[1]

'Happy birthday to our lead player, Jordan Lake! https://t.co/YG3jaC8ZdV'

In [59]:
df_tweet.place[1]

# Cleaning tweets in the dataframe 

#### Lets clean tweets after labeling

In [60]:
# # Clean each tweet in our dataframe using the preprocessor module
# tweet_clean = []
# for tweet in df_tweet['tweet']:
#     cleaned = p.clean(tweet)
#     tweet_clean.append(cleaned)

In [61]:
# # Store as a new dataframe
# df_tweet_clean = df_tweet.copy()
# df_tweet_clean['tweet'] = tweet_clean

In [62]:
#df_tweet_clean.head()

# Save the dataframe as a CSV

In [63]:
# Checking current working directory
import os
os.getcwd()

'C:\\Users\\rapiduser\\Documents\\WaterQualityPerception\\Pierre\\WaterQualityPerceptionMP2020'

In [64]:
# Store as CSV to a relative path
#df_tweet.to_csv(f"..\\..\\Data\\jordanlake_{year_from}_{year_to}.csv")
df_tweet.to_csv(f"..\\..\\Data\\jordanlake_{year_from}_{year_to}links.csv")
#df_tweet_clean.to_csv(f"..\\..\\Data\\utahlake_{year}.csv")
#df_tweet_clean.to_csv("..\\..\\Data\\scraped_tweets_test.csv")